<a href="https://colab.research.google.com/github/gerasimos-matidis/my_icdar/blob/main/segm_w_on_the_fly_augm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Segmentation with on the fly Data Augmentation


<a href="https://colab.research.google.com/github/gerasimos-matidis/my_icdar/blob/main/segm_w_on_the_fly_augm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount google drive and move to the project directory
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/MyDrive/my_icdar

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/my_icdar


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
from gm_networks import unet_model

In [3]:
# Load the initial input and output images for training and validation and expand their dimensions to fit the ImageDataGenerator 
# For inputs: initial_dimensions = (width, height, 3) ---> new_dimensions = (1, width, height, 3)
# For outputs: initial_dimensions = (width, height) ---> new_dimensions = (1, width, height, 1)

x_initial_train = plt.imread('train/101-INPUT.jpg')
y_initial_train = plt.imread('train/101-OUTPUT-GT.png')
x_initial_train = np.expand_dims(x_initial_train, 0) 
y_initial_train = np.expand_dims(y_initial_train, (0, -1)) 

"""
x_initial_valid = plt.imread('validation/201-INPUT.jpg')
y_initial_valid = plt.imread('validation/201-OUTPUT-GT.png')
x_initial_valid = np.expand_dims(x_initial_valid, 0)
y_initial_valid = np.expand_dims(y_initial_valid, (0, -1))
"""

"\nx_initial_valid = plt.imread('validation/201-INPUT.jpg')\ny_initial_valid = plt.imread('validation/201-OUTPUT-GT.png')\nx_initial_valid = np.expand_dims(x_initial_valid, 0)\ny_initial_valid = np.expand_dims(y_initial_valid, (0, -1))\n"

In [4]:
# Define the new size to create images of size (NEW_SIZE, NEW_SIZE), as well as define the number of train and validation images to be created
NEW_SIZE = 512
TRAIN_IMAGES_NUMBER = 450
#VALIDATION_IMAGES_NUMBER = 50

In [5]:
# Concatenate the respective inputs and outputs, in order to ensure that they will undergo the same crop
concatenated_images_train = np.concatenate([x_initial_train, y_initial_train], -1)
#concatenated_images_valid = np.concatenate([x_initial_valid, y_initial_valid], -1)

# Initialize the tensors for the train and validation batches
croped_images_train = np.zeros([TRAIN_IMAGES_NUMBER, NEW_SIZE, NEW_SIZE, 4])
#croped_images_valid = np.zeros([VALIDATION_IMAGES_NUMBER, NEW_SIZE, NEW_SIZE, 4])

# Random croping
for i in range(TRAIN_IMAGES_NUMBER):
    
    t = tf.image.random_crop(concatenated_images_train, size=[1, NEW_SIZE, NEW_SIZE, 4])
    croped_images_train[i] = t[0]
"""
for i in range(VALIDATION_IMAGES_NUMBER):
    
    v = tf.image.random_crop(concatenated_images_valid, size=[1, NEW_SIZE, NEW_SIZE, 4])
    croped_images_valid[i] = v[0]
"""
# Separate the inputs from the outputs to create the final batches
x_train = croped_images_train[:, :, :, :3]
y_train = np.expand_dims(croped_images_train[:, :, :, 3], -1)

"""
x_valid = croped_images_valid[:, :, :, :3]
y_valid = np.expand_dims(croped_images_valid[:, :, :, 3], -1)
"""

'\nx_valid = croped_images_valid[:, :, :, :3]\ny_valid = np.expand_dims(croped_images_valid[:, :, :, 3], -1)\n'

In [6]:
# Create dictionaries with the alterations to be used for the data augmentation operations for inputs and outputs
""""
NOTE: It is important to use exact the same values for both dictionaries (for inputs and outputs). 
The reason why we create 2 dictionaries instead of a common one is because  we want to add the 
preprocessing function for the output masks (This function sets all the pixel values of the mask to 0 or 1. 
While the initial images are binary, after the data augmentation operations, such as rotations and shifts, 
pixels with intermediate values are created due to interpolation)
"""
x_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect')

y_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect', 
    preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype))

In [7]:
BATCH_SIZE = 8
x_datagen = ImageDataGenerator(**x_datagen_args)
y_datagen = ImageDataGenerator(**y_datagen_args)

seed = 17
x_datagen.fit(x_train, augment=True, seed=seed)
y_datagen.fit(y_train, augment=True, seed=seed)

x_generator_train = x_datagen.flow(x_train, seed=seed, batch_size=BATCH_SIZE)
#x_generator_valid = x_datagen.flow(x_valid, seed=seed, batch_size=BATCH_SIZE)
y_generator_train = y_datagen.flow(y_train, seed=seed, batch_size=BATCH_SIZE)
#y_generator_valid = y_datagen.flow(y_valid, seed=seed, batch_size=BATCH_SIZE)

train_generator = zip(x_generator_train, y_generator_train)
#valid_generator = zip(x_generator_valid, y_generator_valid)

In [ ]:
INITIAL_FILTERS = 32
EPOCHS = 50
STEPS_PER_EPOCH = TRAIN_IMAGES_NUMBER // BATCH_SIZE

model = unet_model(input_shape=(NEW_SIZE, NEW_SIZE, 3), initial_filters=INITIAL_FILTERS)
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_generator, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
model_path = f'models/Unet_inputsize_{NEW_SIZE}_epochs{EPOCHS}_stepsperepoch{STEPS_PER_EPOCH}}'
model.save(model_path)

In [ ]:
np.save('models/history.npy', history.history)